# CS246 - Colab 4
## Collaborative Filtering

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 63kB/s 
     |████████████████████████████████| 204kB 42.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=61056ea18f5078bd15174a736ad05ab61f919c17dc2f694e010586ea96ba6adc
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 35.8 MB of archives.
After this 

Now we authenticate a Google Drive client to download the filea we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf = conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-07-31 07:10:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.195.187.253, 54.161.19.10, 35.175.20.97, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.195.187.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  57.9MB/s    in 0.2s    

2020-07-31 07:10:05 (57.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://bf87e22cce96.ngrok.io


### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [8]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [9]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [10]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [18]:
# To compute the number of ratings in the training, test dataset
print(f"There are a total of {training.count()} ratings in the training dataset")
print(f"There are a total of {test.count()} ratings in the test dataset")
print(f"There are a total of {items.count()} movies in the dataset")


There are a total of 80000 ratings in the training dataset
There are a total of 20000 ratings in the test dataset
There are a total of 1682 movies in the dataset


Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [21]:
# import the necessary libraries
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

# Build the recommendation model using ALS on the training data
als = ALS(rank = 10, maxIter = 10, regParam = 0.05, userCol = 'user_id', itemCol = 'item_id', ratingCol = 'rating', coldStartStrategy = 'drop')
model = als.fit(training)

Now compute the RMSE on the test dataset.


In [24]:
# predict the ratings on the test set
predictions = model.transform(test)

# perfrom the model evaluation using RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol = 'rating', predictionCol = 'prediction')
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


Root-mean-square error = 0.9772023825075241


In [31]:
# Build the recommendation model using ALS on the training data
als = ALS(rank = 5, maxIter = 10, regParam = 0.05, userCol = 'user_id', itemCol = 'item_id', ratingCol = 'rating', coldStartStrategy = 'drop')
model = als.fit(training)

# predict the ratings on the test set
predictions = model.transform(test)

# perfrom the model evaluation using RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol = 'rating', predictionCol = 'prediction')
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


Root-mean-square error = 0.959792733047571


In [33]:
reg_list = [0.01, 0.05, 0.1, 0.5]
for reg in reg_list:

  # Build the recommendation model using ALS on the training data
  als = ALS(rank = 5, maxIter = 10, regParam = reg, userCol = 'user_id', itemCol = 'item_id', ratingCol = 'rating', coldStartStrategy = 'drop')
  model = als.fit(training)

  # predict the ratings on the test set
  predictions = model.transform(test)

  # perfrom the model evaluation using RMSE
  evaluator = RegressionEvaluator(metricName="rmse", labelCol = 'rating', predictionCol = 'prediction')
  rmse = evaluator.evaluate(predictions)
  print("Root-mean-square error = " + str(rmse) +" when reg is " + str(reg))


Root-mean-square error = 1.0257683298171587 when reg is 0.01
Root-mean-square error = 0.959792733047571 when reg is 0.05
Root-mean-square error = 0.9334998580325293 when reg is 0.1
Root-mean-square error = 1.0742546999109326 when reg is 0.5


At this point, you can use the trained model to produce the top-K recommendations for each user.

In [27]:
# produce the top-10 recommendations for each user
userRecs = model.recommendForAllUsers(10)

# produce the top-10 user recommendations for each movie
movieRecs = model.recommendForAllItems(10)


DataFrame[user_id: int, recommendations: array<struct<item_id:int,rating:float>>]

In [29]:
userRecs.take(5)

[Row(user_id=471, recommendations=[Row(item_id=962, rating=8.115094184875488), Row(item_id=1167, rating=7.527532577514648), Row(item_id=1207, rating=6.686556339263916), Row(item_id=1155, rating=6.375487327575684), Row(item_id=40, rating=6.283472537994385), Row(item_id=1152, rating=6.275321960449219), Row(item_id=1217, rating=6.259552001953125), Row(item_id=1278, rating=6.231823921203613), Row(item_id=1615, rating=6.2159810066223145), Row(item_id=944, rating=6.018719673156738)]),
 Row(user_id=463, recommendations=[Row(item_id=851, rating=4.9817352294921875), Row(item_id=867, rating=4.9796624183654785), Row(item_id=1643, rating=4.9398112297058105), Row(item_id=916, rating=4.92707633972168), Row(item_id=909, rating=4.909367561340332), Row(item_id=793, rating=4.890866279602051), Row(item_id=1449, rating=4.838788986206055), Row(item_id=1512, rating=4.8054070472717285), Row(item_id=516, rating=4.7797112464904785), Row(item_id=408, rating=4.715842247009277)]),
 Row(user_id=833, recommendation

In [59]:
# Store the top 1 recommendations
userRecs_new = userRecs.select('user_id', expr("recommendations['item_id']")[0].alias('item_id'))

# join the table with the item table
userRecs_new.join(items, userRecs_new.item_id == items.item_id).select(col('user_id'),col('movie').alias("movie_recommendation")).take(10)

[Row(user_id=471, movie_recommendation='Ruby in Paradise (1993)'),
 Row(user_id=463, movie_recommendation='Two or Three Things I Know About Her (1966)'),
 Row(user_id=833, movie_recommendation='Denise Calls Up (1995)'),
 Row(user_id=496, movie_recommendation='Ghost in the Shell (Kokaku kidotai) (1995)'),
 Row(user_id=148, movie_recommendation='Die xue shuang xiong (Killer, The) (1989)'),
 Row(user_id=540, movie_recommendation='Safe (1995)'),
 Row(user_id=392, movie_recommendation='Angel Baby (1995)'),
 Row(user_id=243, movie_recommendation='Angel Baby (1995)'),
 Row(user_id=623, movie_recommendation='Angel Baby (1995)'),
 Row(user_id=737, movie_recommendation='Mina Tannenbaum (1994)')]

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!